In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import json
import sys
import requests
# setting path
sys.path.append('../common')
import utils
import fillInUtils
import limitUtils

SETUP_PATH = 'setup.json'
f = open(SETUP_PATH)
data = json.load(f)

### Fill Django

In [4]:
response = requests.get(data['app']["url"])
content = response.content.decode("utf-8")
len(json.loads(content))

100000

In [7]:
fillInUtils.fill_framework(
    app=data['app'],
    N = data['fill_in_quantity'],
)

http://localhost:8000/blogs/clear/
b'{"Count of Blogs":0}'
200
http://localhost:8000/blogs/fill_in/
100000
iterations: 200
1/200: b'{"Count of Blogs":500}'
1/200: 200
1/200: duration: 1s
1/200: expected end in: 3m 19s
2/200: b'{"Count of Blogs":1000}'
2/200: 200
2/200: duration: 1s
2/200: expected end in: 3m 18s
3/200: b'{"Count of Blogs":1500}'
3/200: 200
3/200: duration: 1s
3/200: expected end in: 3m 17s
4/200: b'{"Count of Blogs":2000}'
4/200: 200
4/200: duration: 1s
4/200: expected end in: 3m 16s
5/200: b'{"Count of Blogs":2500}'
5/200: 200
5/200: duration: 1s
5/200: expected end in: 3m 15s
6/200: b'{"Count of Blogs":3000}'
6/200: 200
6/200: duration: 1s
6/200: expected end in: 3m 14s
7/200: b'{"Count of Blogs":3500}'
7/200: 200
7/200: duration: 1s
7/200: expected end in: 3m 13s
8/200: b'{"Count of Blogs":4000}'
8/200: 200
8/200: duration: 1s
8/200: expected end in: 3m 12s
9/200: b'{"Count of Blogs":4500}'
9/200: 200
9/200: duration: 1s
9/200: expected end in: 3m 11s
10/200: b'{"Co

## Run k6 test script

In [5]:
limitUtils.test_vus_limit(
    result_base_dir=data["test_result_base_dir"],
    test_result_dir_prefix=data["test_result_dir_prefix"],
    script_path=data["script_path"],
    setup_path=SETUP_PATH,
    app=data["app"],
    inital_vus=2052,
    out_file_name="out_fill_in.txt",
    end=2054
)

date and time = 2024-06-01_19-03-47
./out/out_2024-06-01_19-03-47

          /\      |‾‾| /‾‾/   /‾‾/   
     /\  /  \     |  |/  /   /  /    
    /  \/    \    |     (   /   ‾‾\  
   /          \   |  |\  \ |  (‾)  | 
  / __________ \  |__| \__\ \_____/ .io

     execution: local
        script: ../../grafana-k6/script.js
        output: json (./out/out_2024-06-01_19-03-47/django.json), csv (./out/out_2024-06-01_19-03-47/django.csv)

     scenarios: (100.00%) 1 scenario, 2052 max VUs, 1m0s max duration (incl. graceful stop):
              * default: 2052 looping VUs for 30s (gracefulStop: 30s)


running (0m00.4s), 2052/2052 VUs, 0 complete and 0 interrupted iterations
default   [   1% ] 2052 VUs  00.4s/30s

running (0m01.4s), 2052/2052 VUs, 0 complete and 0 interrupted iterations
default   [   5% ] 2052 VUs  01.4s/30s

running (0m02.4s), 2052/2052 VUs, 0 complete and 0 interrupted iterations
default   [   8% ] 2052 VUs  02.4s/30s

running (0m03.4s), 2052/2052 VUs, 0 complete and 0 int

time="2024-06-01T19:05:54+02:00" level=warning msg="No script iterations fully finished, consider making the test duration longer"



running (1m00.3s), 0000/2053 VUs, 0 complete and 2053 interrupted iterations
default ✓ [ 100% ] 2053 VUs  30s

     data_received...: 658 B  11 B/s
     data_sent.......: 177 kB 2.9 kB/s
     vus.............: 2053   min=1588 max=2053
     vus_max.........: 2053   min=2053 max=2053


running (1m00.3s), 0000/2053 VUs, 0 complete and 2053 interrupted iterations
default ✓ [ 100% ] 2053 VUs  30s
Incorrect part for vus = 2053: nan



/home/jan/Dokumenty/Studia/MSem3/Magisterka/sandbox/framework-benchmark/tests/02-limit-django/../common/utils.py:23: RuntimeWarning: invalid value encountered in scalar divide
  incorrect_part = incorrect_status.count() / (correct_status.count() + incorrect_status.count())
